In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns

# Cargar datos
df = pd.read_csv('TCGA-BRCA_log_FPKM.txt', sep="\t", index_col=0)  # Asegurar que la primera columna es índice

# Transponer para que las muestras sean filas y los genes sean columnas
df = df.T

# Convertir a valores numéricos
df = df.apply(pd.to_numeric, errors='coerce')

# Eliminar genes (columnas) con valores NaN
df = df.dropna(axis=1)

# Normalizar los datos
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# Aplicar PCA
pca = PCA(n_components=5)  # Ajusta según la varianza explicada
X_pca = pca.fit_transform(df_scaled)

# Método del codo para determinar clusters
wcss = []
k_values = range(1, 11)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca)
    wcss.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(k_values, wcss, marker='o', linestyle='-')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS (Within-Cluster Sum of Squares)')
plt.title('Método del Codo para K óptimo')
plt.show()

# Definir número óptimo de clusters (ajustar según el gráfico)
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(X_pca)

# Guardar los identificadores de cada cluster en variables separadas
cluster_0 = df.index[df['Cluster'] == 0].tolist()
cluster_1 = df.index[df['Cluster'] == 1].tolist()
cluster_2 = df.index[df['Cluster'] == 2].tolist()

print(cluster_0)
print(cluster_1)
print(cluster_2)

# Visualizar clusters tras PCA
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df['Cluster'], palette='viridis', alpha=0.7)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Clusters tras PCA')
plt.legend(title='Cluster')
plt.show()
